<a href="https://colab.research.google.com/github/HansPiterson/AvatarRobloxChecker/blob/main/gamepass_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🚀 Validasi GamePass V6 (The New API Method)
import requests
import math
from IPython.display import Image, display

# ==========================================
# KONFIGURASI
# ==========================================
USERNAME = "NewWorldOrdeer"
JUMLAH_BELI = 1000

# ==========================================
# LOGIKA PROGRAM
# ==========================================

def get_user_id(username):
    try:
        payload = {"usernames": [username], "excludeBannedUsers": False}
        req = requests.post("https://users.roblox.com/v1/usernames/users", json=payload)
        data = req.json()['data']
        return data[0]['id'] if data else None
    except:
        return None

def scan_with_new_api(user_id, min_netto_target):
    print(f"   ↳ ⚡ Menggunakan New API (apis.roblox.com)...")

    # URL Dasar
    base_url = f"https://apis.roblox.com/game-passes/v1/users/{user_id}/game-passes"

    cursor = "" # StartId (Awalnya kosong)
    has_more_data = True
    page_count = 0

    while has_more_data:
        page_count += 1
        print(f"      📄 Scanning Halaman {page_count}...")

        try:
            # Parameter sesuai instruksi Anda
            params = {
                "count": 100,
                "exclusiveStartId": cursor
            }

            req = requests.get(base_url, params=params)

            # Cek status
            if req.status_code == 403:
                print("      🔒 Akses Ditolak (Inventory Private?).")
                return None
            elif req.status_code != 200:
                print(f"      ❌ Error API: Status {req.status_code}")
                print(f"      Response: {req.text}")
                return None

            data = req.json()

            # Struktur response biasanya: { "gamePasses": [...], "nextPageCursor": ... }
            # Tapi karena cursor manual, kita ambil listnya langsung
            items = data.get('gamePasses', [])

            if not items:
                print("      ⚠️ Tidak ada data GamePass lagi.")
                break

            # 1. Loop Item di halaman ini
            for item in items:
                # Cek harga
                price = item.get('price')

                if price:
                    netto = math.floor(price * 0.7)

                    # Validasi
                    if netto >= min_netto_target:
                        # Sukses!
                        # Ambil gambar thumbnail (karena API ini mungkin tidak kasih full URL gambar)
                        # Tapi kita cek dulu kalau-kalau ada field iconUrl
                        image_url = ""
                        if 'iconId' in item:
                             thumb_url = f"https://thumbnails.roblox.com/v1/game-passes/icons?gamePassIds={item['gamePassId']}&size=150x150&format=Png&isCircular=false"
                             image_url = requests.get(thumb_url).json()['data'][0]['imageUrl']

                        return {
                            "name": item.get('name'),
                            "price": price,
                            "netto": netto,
                            "id": item.get('gamePassId'),
                            "image": image_url,
                            "url": f"https://www.roblox.com/game-pass/{item.get('gamePassId')}"
                        }

            # 2. Setup Cursor untuk halaman selanjutnya
            # "give it the last gamepass id your response returns as the next startId"
            last_item = items[-1]
            next_cursor = last_item.get('gamePassId')

            if next_cursor:
                cursor = next_cursor # Update cursor
            else:
                has_more_data = False # Stop jika tidak ada ID

        except Exception as e:
            print(f"      ❌ Error Loop: {e}")
            break

    return None

def smart_validator_v6(username, amount):
    min_sell = math.ceil(amount / 0.7)

    print(f"VALIDASI GAMEPASS V6 (NEW ENDPOINT)")
    print(f"User: {username} |  Target: {amount} R$ (GP Min: {min_sell} R$)")
    print("="*60)

    # 1. Get User ID
    user_id = get_user_id(username)
    if not user_id:
        print("❌ User tidak ditemukan.")
        return
    print(f"User ID: {user_id}")

    # 2. Scan menggunakan API Baru
    found_pass = scan_with_new_api(user_id, amount)

    # 3. Hasil
    print("\n" + "="*60)
    if found_pass:
        print("200 OK")
        print(f"🎫   : {found_pass['name']}")
        print(f"price  : {found_pass['price']} R$ (Netto: {found_pass['netto']})")
        print(f"Link   : {found_pass['url']}")
        if found_pass.get('image'):
            display(Image(url=found_pass['image']))
    else:
        print("❌ GAGAL MENEMUKAN GAMEPASS.")
        print(f"Pastikan inventory PUBLIC dan GamePass seharga {min_sell} R$ sudah dibuat.")

# Jalankan
smart_validator_v6(USERNAME, JUMLAH_BELI)

VALIDASI GAMEPASS V6 (NEW ENDPOINT)
User: NewWorldOrdeer |  Target: 1000 R$ (GP Min: 1429 R$)
User ID: 1618224997
   ↳ ⚡ Menggunakan New API (apis.roblox.com)...
      📄 Scanning Halaman 1...

200 OK
🎫   : tester
price  : 1430 R$ (Netto: 1000)
Link   : https://www.roblox.com/game-pass/1631004908
